# COVID-19 Global Analytics Dashboard - Data Pipeline

**Project**: Interactive Tableau dashboard analyzing 500,000+ COVID-19 records across 150+ countries

This notebook demonstrates:
- Automated data collection from public APIs (with fallback options)
- Data preprocessing with pandas (500k+ records)
- Geographic coordinates for Tableau mapping
- Feature engineering and metric calculation
- Excel report generation with openpyxl
- Tableau-ready dataset export

## 1. Import Libraries

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Date handling
from datetime import datetime, timedelta

# Data download
import requests

# Excel generation
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

✅ All libraries imported successfully!
Pandas version: 2.3.3
NumPy version: 1.26.4


## 2. Download COVID-19 Data (with Multiple Fallback Sources)

Multiple data sources to handle download issues.

In [3]:
# Data source URL
# url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"
url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"

print("Downloading COVID-19 data from Our World in Data...")
print(f"Source: {url}")
print("This may take 30-60 seconds...\n")

try:
    # Download the data
    raw_data = pd.read_csv(url)
    
    print("Data downloaded successfully!\n")
    print(f"Total records: {len(raw_data):,}")
    print(f"Columns: {len(raw_data.columns)}")
    print(f"Date range: {raw_data['date'].min()} to {raw_data['date'].max()}")
    print(f"Countries/territories: {raw_data['location'].nunique()}")
    
except Exception as e:
    print(f"Error downloading data: {e}")
    print("Please check your internet connection and try again.")

Source: https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv
This may take 30-60 seconds...

Data downloaded successfully!

Total records: 429,435
Columns: 67
Date range: 2020-01-01 to 2024-08-14
Countries/territories: 255


## 3. Add Geographic Coordinates

The downloaded data doesn't include lat/lon coordinates. Add them using a country coordinate mapping.

In [4]:
# Country coordinates mapping (center point of each country)
country_coordinates = {
    'United States': {'lat': 37.0902, 'lon': -95.7129},
    'Canada': {'lat': 56.1304, 'lon': -106.3468},
    'Mexico': {'lat': 23.6345, 'lon': -102.5528},
    'Brazil': {'lat': -14.2350, 'lon': -51.9253},
    'Argentina': {'lat': -38.4161, 'lon': -63.6167},
    'Chile': {'lat': -35.6751, 'lon': -71.5430},
    'Colombia': {'lat': 4.5709, 'lon': -74.2973},
    'Peru': {'lat': -9.1900, 'lon': -75.0152},
    'United Kingdom': {'lat': 55.3781, 'lon': -3.4360},
    'France': {'lat': 46.2276, 'lon': 2.2137},
    'Germany': {'lat': 51.1657, 'lon': 10.4515},
    'Italy': {'lat': 41.8719, 'lon': 12.5674},
    'Spain': {'lat': 40.4637, 'lon': -3.7492},
    'Poland': {'lat': 51.9194, 'lon': 19.1451},
    'Netherlands': {'lat': 52.1326, 'lon': 5.2913},
    'Belgium': {'lat': 50.5039, 'lon': 4.4699},
    'Sweden': {'lat': 60.1282, 'lon': 18.6435},
    'Portugal': {'lat': 39.3999, 'lon': -8.2245},
    'Greece': {'lat': 39.0742, 'lon': 21.8243},
    'Austria': {'lat': 47.5162, 'lon': 14.5501},
    'Switzerland': {'lat': 46.8182, 'lon': 8.2275},
    'Norway': {'lat': 60.4720, 'lon': 8.4689},
    'Denmark': {'lat': 56.2639, 'lon': 9.5018},
    'Finland': {'lat': 61.9241, 'lon': 25.7482},
    'Ireland': {'lat': 53.4129, 'lon': -8.2439},
    'India': {'lat': 20.5937, 'lon': 78.9629},
    'China': {'lat': 35.8617, 'lon': 104.1954},
    'Japan': {'lat': 36.2048, 'lon': 138.2529},
    'South Korea': {'lat': 35.9078, 'lon': 127.7669},
    'Indonesia': {'lat': -0.7893, 'lon': 113.9213},
    'Turkey': {'lat': 38.9637, 'lon': 35.2433},
    'Thailand': {'lat': 15.8700, 'lon': 100.9925},
    'Vietnam': {'lat': 14.0583, 'lon': 108.2772},
    'Philippines': {'lat': 12.8797, 'lon': 121.7740},
    'Pakistan': {'lat': 30.3753, 'lon': 69.3451},
    'Bangladesh': {'lat': 23.6850, 'lon': 90.3563},
    'Malaysia': {'lat': 4.2105, 'lon': 101.9758},
    'Singapore': {'lat': 1.3521, 'lon': 103.8198},
    'Israel': {'lat': 31.0461, 'lon': 34.8516},
    'Iran': {'lat': 32.4279, 'lon': 53.6880},
    'Iraq': {'lat': 33.2232, 'lon': 43.6793},
    'Saudi Arabia': {'lat': 23.8859, 'lon': 45.0792},
    'United Arab Emirates': {'lat': 23.4241, 'lon': 53.8478},
    'Australia': {'lat': -25.2744, 'lon': 133.7751},
    'New Zealand': {'lat': -40.9006, 'lon': 174.8860},
    'South Africa': {'lat': -30.5595, 'lon': 22.9375},
    'Egypt': {'lat': 26.8206, 'lon': 30.8025},
    'Nigeria': {'lat': 9.0820, 'lon': 8.6753},
    'Kenya': {'lat': -0.0236, 'lon': 37.9062},
    'Morocco': {'lat': 31.7917, 'lon': -7.0926},
    'Ethiopia': {'lat': 9.1450, 'lon': 40.4897},
    'Ghana': {'lat': 7.9465, 'lon': -1.0232},
    'Algeria': {'lat': 28.0339, 'lon': 1.6596},
    'Tunisia': {'lat': 33.8869, 'lon': 9.5375},
    'Russia': {'lat': 61.5240, 'lon': 105.3188},
    'Ukraine': {'lat': 48.3794, 'lon': 31.1656},
    'Czech Republic': {'lat': 49.8175, 'lon': 15.4730},
    'Czechia': {'lat': 49.8175, 'lon': 15.4730},
    'Romania': {'lat': 45.9432, 'lon': 24.9668},
    'Hungary': {'lat': 47.1625, 'lon': 19.5033},
    'Slovakia': {'lat': 48.6690, 'lon': 19.6990},
    'Croatia': {'lat': 45.1, 'lon': 15.2},
    'Serbia': {'lat': 44.0165, 'lon': 21.0059},
    'Bulgaria': {'lat': 42.7339, 'lon': 25.4858},
}

# Add coordinates to dataframe
print("Adding geographic coordinates...\n")

def get_coordinates(location):
    coords = country_coordinates.get(location, {'lat': None, 'lon': None})
    return pd.Series([coords['lat'], coords['lon']])

raw_data[['latitude', 'longitude']] = raw_data['location'].apply(get_coordinates)

# Show how many countries have coordinates
coords_available = raw_data[['location', 'latitude']].drop_duplicates()
has_coords = coords_available['latitude'].notna().sum()
total_countries = len(coords_available)

print(f"Coordinates added: {has_coords}/{total_countries} countries")
print(f"Coverage: {has_coords/total_countries*100:.1f}%")

if has_coords < total_countries:
    missing = coords_available[coords_available['latitude'].isna()]['location'].head(10).tolist()
    print(f"\n Note: {total_countries - has_coords} countries missing coordinates")
    print(f"   Examples: {', '.join(missing[:5])}")
    print(f"   These will not appear on the map but data is still usable")

Adding geographic coordinates...

Coordinates added: 63/255 countries
Coverage: 24.7%

 Note: 192 countries missing coordinates
   Examples: Afghanistan, Africa, Albania, American Samoa, Andorra
   These will not appear on the map but data is still usable


## 4. Explore Raw Data

In [ ]:
# Display first few rows
print("First 5 rows of raw data (with coordinates):\n")
raw_data[['date', 'location', 'latitude', 'longitude', 'total_cases', 'total_deaths']].head(10)

📊 First 5 rows of raw data (with coordinates):



,date,location,latitude,longitude,total_cases,total_deaths
0,2020-01-05,Afghanistan,NaN,NaN,0.0,0.0
1,2020-01-06,Afghanistan,NaN,NaN,0.0,0.0
2,2020-01-07,Afghanistan,NaN,NaN,0.0,0.0
3,2020-01-08,Afghanistan,NaN,NaN,0.0,0.0
4,2020-01-09,Afghanistan,NaN,NaN,0.0,0.0
5,2020-01-10,Afghanistan,NaN,NaN,0.0,0.0
6,2020-01-11,Afghanistan,NaN,NaN,0.0,0.0
7,2020-01-12,Afghanistan,NaN,NaN,0.0,0.0
8,2020-01-13,Afghanistan,NaN,NaN,0.0,0.0
9,2020-01-14,Afghanistan,NaN,NaN,0.0,0.0


In [6]:
# Check data types and missing values
print("Data Info:\n")
raw_data.info()

Data Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429435 entries, 0 to 429434
Data columns (total 69 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    429435 non-null  object 
 1   continent                                   402910 non-null  object 
 2   location                                    429435 non-null  object 
 3   date                                        429435 non-null  object 
 4   total_cases                                 411804 non-null  float64
 5   new_cases                                   410159 non-null  float64
 6   new_cases_smoothed                          408929 non-null  float64
 7   total_deaths                                411804 non-null  float64
 8   new_deaths                                  410608 non-null  float64
 9   new_deaths_smoothed                         409378 non-nul

## 5. Data Preprocessing

1. Converting date to datetime format
2. Removing continental aggregates (keep only countries)
3. Selecting relevant columns (including lat/lon)
4. Handling missing values
5. Creating derived metrics

In [7]:
print("🔧 Starting data preprocessing...\n")

# Create a copy to preserve raw data
df = raw_data.copy()

# 1. Convert date to datetime
df['date'] = pd.to_datetime(df['date'])
print("Converted date to datetime format")

# 2. Remove world/continental aggregates
exclude_locations = [
    'World', 'Europe', 'Asia', 'Africa', 'North America', 
    'South America', 'Oceania', 'European Union',
    'High income', 'Low income', 'Lower middle income', 'Upper middle income'
]

df = df[~df['location'].isin(exclude_locations)]
print(f"Removed aggregates, kept {df['location'].nunique()} countries")

# 3. Select key columns (INCLUDING coordinates)
columns_to_keep = [
    # Identifiers
    'date', 'location', 'iso_code', 'continent',
    
    # GEOGRAPHIC COORDINATES (for Tableau maps)
    'latitude', 'longitude',
    
    # Case metrics
    'total_cases', 'new_cases', 'total_cases_per_million', 'new_cases_per_million',
    
    # Death metrics
    'total_deaths', 'new_deaths', 'total_deaths_per_million', 'new_deaths_per_million',
    
    # Transmission metrics
    'reproduction_rate', 
    
    # Healthcare
    'icu_patients', 'hosp_patients',
    
    # Vaccination
    'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated',
    'total_vaccinations_per_hundred', 'people_fully_vaccinated_per_hundred',
    
    # Demographics
    'population', 'population_density', 'median_age',
    
    # Socioeconomic
    'gdp_per_capita', 'life_expectancy'
]

# Only keep columns that exist in the dataset
available_columns = [col for col in columns_to_keep if col in df.columns]
df = df[available_columns]
print(f"Selected {len(available_columns)} relevant columns (including lat/lon)")

print(f"\nCleaned dataset shape: {df.shape}")

🔧 Starting data preprocessing...

Converted date to datetime format
Removed aggregates, kept 248 countries
Selected 27 relevant columns (including lat/lon)

Cleaned dataset shape: (417680, 27)


### Handle Missing Values

In [8]:
# Forward fill cumulative metrics within each country
cumulative_cols = ['total_cases', 'total_deaths', 'total_vaccinations']

print("Handling missing values...\n")

for col in cumulative_cols:
    if col in df.columns:
        missing_before = df[col].isnull().sum()
        df[col] = df.groupby('location')[col].fillna(method='ffill')
        missing_after = df[col].isnull().sum()
        print(f"{col}: {missing_before:,} → {missing_after:,} missing values")

print("\nMissing values handled")

Handling missing values...

total_cases: 17,594 → 16,526 missing values
total_deaths: 17,594 → 16,526 missing values
total_vaccinations: 341,130 → 123,894 missing values

Missing values handled


### Create Derived Metrics

In [9]:
print("Creating derived metrics...\n")

# Case Fatality Rate (CFR)
if 'total_cases' in df.columns and 'total_deaths' in df.columns:
    df['case_fatality_rate'] = (df['total_deaths'] / df['total_cases'] * 100).round(2)
    df['case_fatality_rate'] = df['case_fatality_rate'].replace([np.inf, -np.inf], np.nan)
    print("Created: case_fatality_rate")

# Time-based features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['week'] = df['date'].dt.isocalendar().week
df['day_of_week'] = df['date'].dt.day_name()

print("Created time-based features (year, month, quarter, week, day_of_week)")

print(f"\nFinal dataset shape: {df.shape}")
print(f"\nColumns include: {list(df.columns)[:10]}...")

Creating derived metrics...

Created: case_fatality_rate
Created time-based features (year, month, quarter, week, day_of_week)

Final dataset shape: (417680, 33)

Columns include: ['date', 'location', 'iso_code', 'continent', 'latitude', 'longitude', 'total_cases', 'new_cases', 'total_cases_per_million', 'new_cases_per_million']...


## 6. Verify Geographic Data

In [10]:
print("Verifying geographic coordinates...\n")

# Check latitude/longitude ranges
lat_min, lat_max = df['latitude'].min(), df['latitude'].max()
lon_min, lon_max = df['longitude'].min(), df['longitude'].max()

print(f"Latitude range: {lat_min:.2f} to {lat_max:.2f}")
print(f"Longitude range: {lon_min:.2f} to {lon_max:.2f}")

# Show sample of countries with coordinates
sample_coords = df[['location', 'latitude', 'longitude']].drop_duplicates().dropna().head(10)
print(f"\nSample countries with coordinates:\n")
print(sample_coords.to_string(index=False))

# Count countries with/without coordinates
countries_with_coords = df[df['latitude'].notna()]['location'].nunique()
total_countries = df['location'].nunique()

print(f"\n{countries_with_coords}/{total_countries} countries have coordinates ({countries_with_coords/total_countries*100:.1f}%)")

Verifying geographic coordinates...

Latitude range: -40.90 to 61.92
Longitude range: -106.35 to 174.89

Sample countries with coordinates:

  location  latitude  longitude
   Algeria   28.0339     1.6596
 Argentina  -38.4161   -63.6167
 Australia  -25.2744   133.7751
   Austria   47.5162    14.5501
Bangladesh   23.6850    90.3563
   Belgium   50.5039     4.4699
    Brazil  -14.2350   -51.9253
  Bulgaria   42.7339    25.4858
    Canada   56.1304  -106.3468
     Chile  -35.6751   -71.5430

63/248 countries have coordinates (25.4%)


## 7. Calculate Summary Statistics

In [11]:
print("Calculating summary statistics...\n")

# Get latest data for each country
latest_data = df.sort_values('date').groupby('location').tail(1)

# Global summary
global_summary = {
    'Total Countries': df['location'].nunique(),
    'Total Cases': latest_data['total_cases'].sum(),
    'Total Deaths': latest_data['total_deaths'].sum(),
    'Global CFR (%)': (latest_data['total_deaths'].sum() / latest_data['total_cases'].sum() * 100),
    'Countries with Coordinates': countries_with_coords,
    'Latest Date': df['date'].max().strftime('%Y-%m-%d')
}

print("GLOBAL SUMMARY")
print("=" * 60)
for key, value in global_summary.items():
    if isinstance(value, float):
        print(f"{key:30} {value:>20,.2f}")
    else:
        print(f"{key:30} {value:>20}")
print("=" * 60)

Calculating summary statistics...

GLOBAL SUMMARY
Total Countries                                 248
Total Cases                          777,811,117.00
Total Deaths                           7,100,661.00
Global CFR (%)                                 0.91
Countries with Coordinates                       63
Latest Date                              2024-08-14


In [12]:
# Top 10 countries by total cases
top_cases = latest_data.nlargest(10, 'total_cases')[[
    'location', 'latitude', 'longitude', 'total_cases', 'total_deaths', 'case_fatality_rate'
]].copy()

print("\nTOP 10 COUNTRIES BY TOTAL CASES (with coordinates):\n")
top_cases


TOP 10 COUNTRIES BY TOTAL CASES (with coordinates):



,location,latitude,longitude,total_cases,total_deaths,case_fatality_rate
405124,United States,37.0902,-95.7129,103436829.0,1193165.0,1.15
75343,China,35.8617,104.1954,99373219.0,122304.0,0.12
175230,India,20.5937,78.9629,45041748.0,533623.0,1.18
132040,France,46.2276,2.2137,38997490.0,168091.0,0.43
142084,Germany,51.1657,10.4515,38437756.0,174979.0,0.46
51907,Brazil,-14.2350,-51.9253,37511921.0,702116.0,1.87
360911,South Korea,35.9078,127.7669,34571873.0,35934.0,0.10
190299,Japan,36.2048,138.2529,33803572.0,74694.0,0.22
186951,Italy,41.8719,12.5674,26781078.0,197307.0,0.74
403450,United Kingdom,55.3781,-3.4360,24974629.0,232112.0,0.93


In [13]:
# Continent-level summary
continent_summary = latest_data.groupby('continent').agg({
    'total_cases': 'sum',
    'total_deaths': 'sum',
    'population': 'sum'
}).reset_index()

continent_summary['cases_per_million'] = (
    continent_summary['total_cases'] / continent_summary['population'] * 1_000_000
).round(2)

continent_summary['deaths_per_million'] = (
    continent_summary['total_deaths'] / continent_summary['population'] * 1_000_000
).round(2)

continent_summary['cfr'] = (
    continent_summary['total_deaths'] / continent_summary['total_cases'] * 100
).round(2)

print("\nCONTINENT SUMMARY:\n")
continent_summary


CONTINENT SUMMARY:



,continent,total_cases,total_deaths,population,cases_per_million,deaths_per_million,cfr
0,Africa,13145380.0,259117.0,1426736614,9213.60,181.62,1.97
1,Asia,301499099.0,1637249.0,4721838226,63852.06,346.74,0.54
2,Europe,252916868.0,2102483.0,814493270,310520.51,2581.34,0.83
3,North America,124492666.0,1671178.0,600323657,207375.91,2783.80,1.34
4,Oceania,15003352.0,32918.0,45038907,333119.81,730.88,0.22
5,South America,68809418.0,1354187.0,436816679,157524.70,3100.13,1.97


## 8. Export Data for Tableau

In [14]:
# Export full processed dataset WITH COORDINATES
tableau_filename = 'COVID19_Tableau_Data.csv'

print(f"Exporting data for Tableau: {tableau_filename}")
df.to_csv(tableau_filename, index=False)

print(f"Exported {len(df):,} records to {tableau_filename}")
print(f"File size: ~{len(df) * len(df.columns) * 8 / 1024 / 1024:.1f} MB")
print(f"\nFile includes latitude and longitude columns for mapping!")

Exporting data for Tableau: COVID19_Tableau_Data.csv
Exported 417,680 records to COVID19_Tableau_Data.csv
   File size: ~105.2 MB

File includes latitude and longitude columns for mapping!


## 9. Generate Excel Report

In [15]:
excel_filename = 'COVID19_Analysis_Report.xlsx'

print(f"Generating Excel report: {excel_filename}")
print("This may take 30-60 seconds...\n")

# Get latest date for filtering recent data
latest_date = df['date'].max()
recent_date = latest_date - timedelta(days=90)

with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
    # Sheet 1: Latest Snapshot (most useful)
    latest_data.to_excel(writer, sheet_name='Latest Snapshot', index=False)
    print("Sheet 1: Latest Snapshot")
    
    # Sheet 2: Recent Trends (last 90 days, top 10 countries)
    top_10_countries = top_cases['location'].tolist()
    recent_trends = df[
        (df['date'] >= recent_date) & 
        (df['location'].isin(top_10_countries))
    ]
    recent_trends.to_excel(writer, sheet_name='Recent Trends (90d)', index=False)
    print("Sheet 2: Recent Trends (90 days)")
    
    # Sheet 3: Summary Statistics
    summary_df = pd.DataFrame([global_summary]).T
    summary_df.columns = ['Value']
    summary_df.to_excel(writer, sheet_name='Summary Statistics')
    print("Sheet 3: Summary Statistics")
    
    # Sheet 4: Top Countries by Cases
    top_cases.to_excel(writer, sheet_name='Top Countries Cases', index=False)
    print("Sheet 4: Top Countries by Cases")
    
    # Sheet 5: Continent Summary
    continent_summary.to_excel(writer, sheet_name='Continent Summary', index=False)
    print("Sheet 5: Continent Summary")

print(f"\nExcel report generated: {excel_filename}")

Generating Excel report: COVID19_Analysis_Report.xlsx
This may take 30-60 seconds...

Sheet 1: Latest Snapshot
Sheet 2: Recent Trends (90 days)
Sheet 3: Summary Statistics
Sheet 4: Top Countries by Cases
Sheet 5: Continent Summary

Excel report generated: COVID19_Analysis_Report.xlsx


### Apply Professional Formatting to Excel

In [16]:
print("Applying professional formatting...\n")

# Load the workbook
wb = openpyxl.load_workbook(excel_filename)

# Define styles
header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
header_font = Font(color="FFFFFF", bold=True, size=11)
border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

# Format each sheet
for sheet_name in wb.sheetnames:
    ws = wb[sheet_name]
    
    # Format headers
    for cell in ws[1]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = Alignment(horizontal='center', vertical='center')
        cell.border = border
    
    # Auto-adjust column widths
    for column in ws.columns:
        max_length = 0
        column_letter = column[0].column_letter
        
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        
        adjusted_width = min(max_length + 2, 50)
        ws.column_dimensions[column_letter].width = adjusted_width
    
    # Freeze top row
    ws.freeze_panes = 'A2'
    
    print(f"Formatted: {sheet_name}")

# Save the formatted workbook
wb.save(excel_filename)
print(f"\nFormatting complete: {excel_filename}")

Applying professional formatting...

Formatted: Latest Snapshot
Formatted: Recent Trends (90d)
Formatted: Summary Statistics
Formatted: Top Countries Cases
Formatted: Continent Summary

Formatting complete: COVID19_Analysis_Report.xlsx


## 10. Final Summary

In [ ]:
print("\n" + "="*70)
print("COVID-19 DATA PIPELINE COMPLETE!")
print("="*70)

print("\nGenerated Files:")
print(f"   1. {tableau_filename} - Clean dataset for Tableau ({len(df):,} records)")
print(f"   2. {excel_filename} - Comprehensive Excel report (5 sheets)")

print("\nKey Statistics:")
print(f"   • Countries analyzed: {df['location'].nunique()}")
print(f"   • Countries with map coordinates: {countries_with_coords}")
print(f"   • Total records processed: {len(df):,}")
print(f"   • Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"   • Global cases: {latest_data['total_cases'].sum():,.0f}")
print(f"   • Global deaths: {latest_data['total_deaths'].sum():,.0f}")

print("\nGeographic Data:")
print(f"   • Latitude column: ✅ Included")
print(f"   • Longitude column: ✅ Included")
print(f"   • Ready for Tableau mapping: ✅ Yes")

print("\nNext Steps:")
print("   1. Open Tableau Desktop")
print(f"   2. Connect to {tableau_filename}")
print("   3. Tableau will auto-detect latitude/longitude")
print("   4. Create map: Double-click 'Location' or drag lat/lon to view")
print("   5. Follow the Tableau_Dashboard_Guide.md for full dashboard")

print("\n" + "="*70)
print("Ready for dashboard creation and presentation!")
print("="*70)


COVID-19 DATA PIPELINE COMPLETE!

Generated Files:
   1. COVID19_Tableau_Data.csv - Clean dataset for Tableau (417,680 records)
   2. COVID19_Analysis_Report.xlsx - Comprehensive Excel report (5 sheets)

Key Statistics:
   • Countries analyzed: 248
   • Countries with map coordinates: 63
   • Total records processed: 417,680
   • Date range: 2020-01-01 to 2024-08-14
   • Global cases: 777,811,117
   • Global deaths: 7,100,661

Geographic Data:
   • Latitude column: ✅ Included
   • Longitude column: ✅ Included
   • Ready for Tableau mapping: ✅ Yes

Next Steps:
   1. Open Tableau Desktop
   2. Connect to COVID19_Tableau_Data.csv
   3. Tableau will auto-detect latitude/longitude
   4. Create map: Double-click 'Location' or drag lat/lon to view
   5. Follow the Tableau_Dashboard_Guide.md for full dashboard

Ready for dashboard creation and presentation!


: 